In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import config
from derma.dataset import Derma
from derma.architecture import InvertedResidual
from derma.experiment import train_experiment, load_experiment

In [ ]:
# PROBLEM DEFINITION (bucle casos?)

# problem_name = 'block_setting_classes_size-hairs_weighted-classes'
problem_name = 'CA_irv00_2_128_raw_weighted'
dataset_dir = os.path.join(config.DATASET_DIR,'Train','0128-0128_raw')
#problem_name = 'Mn_irv00_2_4000-6000_raw_weighted' 
#dataset_dir = os.path.join(config.DATASET_DIR,'4000-6000_raw')

CoordAtt = False
Weighted_sampling = True

dropout = 0.2 # original is 0.2

from torchvision import transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128))
])
# MODEL
inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

# Original setting for mobilenet v2 (https://github.com/pytorch/vision/blob/main/torchvision/models/mobilenetv2.py)

In [ ]:
# Loading dataset
from derma.experiment import problem_def
train_loader, val_loader, test_loader = problem_def(dataset_dir,Weighted_sampling)

In [ ]:
from derma.dataset import Derma
from derma.architecture import InvertedResidual
from derma.utils import train
import torch
from torchvision.models import MobileNetV2
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
log_dir = os.path.join('log',problem_name) 
model_dir = os.path.join('models',problem_name+'.pt')

if CoordAtt:
    model = MobileNetV2(num_classes=2, inverted_residual_setting=inverted_residual_setting, block=InvertedResidual,dropout=dropout)
else:
    model = MobileNetV2(num_classes=2, inverted_residual_setting=inverted_residual_setting,dropout=dropout) # standard MobileNetV2
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
tb_writer = SummaryWriter(log_dir=log_dir)
train(model, train_loader, optimizer, tb_writer, criterion=torch.nn.CrossEntropyLoss(), n_epoch=10)
torch.save(model.state_dict(), model_dir)

In [ ]:
#train_experiment(dataset_dir,problem_name,Weighted_sampling,CoordAtt,inverted_residual_setting)
#train_experiment(dataset_dir,problem_name,Weighted_sampling,CoordAtt,inverted_residual_setting,transform=transform)


In [ ]:
from derma.experiment import test_experiment
metrics = test_experiment(model,test_loader)